In [2]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import torch.nn.functional as F
import csv
import os

C:\Users\dleir\miniconda3\envs\tfm\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
df = pd.read_csv('poems_cleaned.csv')
df.dropna(axis=0, inplace=True)
test_set = df.sample(n = 200)
df = df.loc[~df.index.isin(test_set.index)]

In [4]:
test_set = test_set.reset_index()
df = df.reset_index()

In [5]:
test_set['True_end_poems'] = test_set['poem'].str.split().str[-20:].apply(' '.join)
test_set['poem'] = test_set['poem'].str.split().str[:-20].apply(' '.join)

In [6]:
class SpanishPoems(Dataset):  
    def __init__(self, control_code, truncate=False, gpt2_type="DeepESP/gpt2-spanish", max_length=1024):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.poems = []

        for row in df['poem']:
          self.poems.append(torch.tensor(
                self.tokenizer.encode(f"<|{control_code}|>{row[:max_length]}<|endoftext|>")
            ))               
        if truncate:
            self.poems = self.poems[:20000]
        self.poems_count = len(self.poems)
        
    def __len__(self):
        return self.poems_count

    def __getitem__(self, item):
        return self.poems[item]
    
dataset = SpanishPoems(df['poem'], truncate=True, gpt2_type="DeepESP/gpt2-spanish") 

Downloading: 100%|███████████████████████████████████████████████████████████████████| 821k/821k [00:00<00:00, 884kB/s]
Downloading: 100%|██████████████████████████████████████████████████████████████████| 487k/487k [00:00<00:00, 1.13MB/s]
Downloading: 100%|█████████████████████████████████████████████████████████████████████| 914/914 [00:00<00:00, 911kB/s]


In [7]:
tokenizer = GPT2Tokenizer.from_pretrained('DeepESP/gpt2-spanish')
model = GPT2LMHeadModel.from_pretrained('DeepESP/gpt2-spanish')

#Accumulated batch size (since GPT2 is so big)
def pack_tensor(new_tensor, packed_tensor, max_seq_len):
    if packed_tensor is None:
        return new_tensor, True, None
    if new_tensor.size()[1] + packed_tensor.size()[1] > max_seq_len:
        return packed_tensor, False, new_tensor
    else:
        packed_tensor = torch.cat([new_tensor, packed_tensor[:, 1:]], dim=1)
        return packed_tensor, True, None

Downloading: 100%|██████████████████████████████████████████████████████████████████| 249M/249M [01:14<00:00, 3.51MB/s]


In [8]:
def train(
    dataset, model, tokenizer,
    batch_size=16, epochs=5, lr=2e-5,
    max_seq_len=400, warmup_steps=200,
    gpt2_type="gpt2", output_dir="./spanish_poems_model_0.2", output_prefix="wreckgar",
    test_mode=False,save_model_on_epoch=False,
):
    acc_steps = 100
    device=torch.device("cuda")
    model = model.cuda()
    model.train()

    optimizer = AdamW(model.parameters(), lr=lr)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=-1
    )

    train_dataloader = DataLoader(dataset, batch_size=1, shuffle=True)
    loss=0
    accumulating_batch_count = 0
    input_tensor = None

    for epoch in range(epochs):

        print(f"Training epoch {epoch}")
        print(loss)
        for idx, entry in tqdm(enumerate(train_dataloader)):
            (input_tensor, carry_on, remainder) = pack_tensor(entry, input_tensor, 768)

            if carry_on and idx != len(train_dataloader) - 1:
                continue

            input_tensor = input_tensor.to(device)
            outputs = model(input_tensor, labels=input_tensor)
            loss = outputs[0]
            loss.backward()

            if (accumulating_batch_count % batch_size) == 0:
                optimizer.step()
                scheduler.step()
                optimizer.zero_grad()
                model.zero_grad()

            accumulating_batch_count += 1
            input_tensor = None
        if save_model_on_epoch:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch}.pt"),
            )
    return model

In [9]:
model = train(dataset, model, tokenizer)

C:\Users\dleir\miniconda3\envs\tfm\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Training epoch 0
0


20000it [16:07, 20.67it/s]


Training epoch 1
tensor(1.5059, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [22:33, 14.78it/s]


...

20000it [2:32:29,  2.19it/s] 


Training epoch 3
tensor(0.6854, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [15:28, 21.54it/s]


Training epoch 4
tensor(1.2472, device='cuda:0', grad_fn=<NllLossBackward0>)


20000it [15:29, 21.53it/s]


In [10]:
torch.save(model, "spanish_poems_model.pt")

In [11]:
prueba = """
No sé, dicen que es invierno
y que fuera está nevando.
Pero aquí sigue siendo verano.

Tú que te coronaste en Cotos sin necesidad de vestido,
o de trampas en papel de efecto retardado,
querías saber lo que me hizo el sol en el Atlántico
pero tú te quemaste la espalda
"""

In [12]:
def generate(
    model,
    tokenizer,
    prompt,
    entry_count=10,
    entry_length=30, #maximum number of words
    top_p=0.8,
    temperature=1.,
):
    model.eval()
    generated_num = 0
    generated_list = []

    filter_value = -float("Inf")

    with torch.no_grad():

        for entry_idx in trange(entry_count):

            entry_finished = False
            generated = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0)

            for i in range(entry_length):
                outputs = model(generated, labels=generated)
                loss, logits = outputs[:2]
                logits = logits[:, -1, :] / (temperature if temperature > 0 else 1.0)

                sorted_logits, sorted_indices = torch.sort(logits, descending=True)
                cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

                sorted_indices_to_remove = cumulative_probs > top_p
                sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[
                    ..., :-1
                ].clone()
                sorted_indices_to_remove[..., 0] = 0

                indices_to_remove = sorted_indices[sorted_indices_to_remove]
                logits[:, indices_to_remove] = filter_value

                next_token = torch.multinomial(F.softmax(logits, dim=-1), num_samples=1)
                generated = torch.cat((generated, next_token), dim=1)

                if next_token in tokenizer.encode("<|endoftext|>"):
                    entry_finished = True

                if entry_finished:

                    generated_num = generated_num + 1

                    output_list = list(generated.squeeze().numpy())
                    output_text = tokenizer.decode(output_list)
                    generated_list.append(output_text)
                    break
            
            if not entry_finished:
              output_list = list(generated.squeeze().numpy())
              output_text = f"{tokenizer.decode(output_list)}<|endoftext|>" 
              generated_list.append(output_text)
                
    return generated_list

In [13]:
def text_generation(text, temperature=1., top_p=0.8, entry_length=30):
  x = generate(model.to('cpu'), tokenizer, text, entry_count=1, temperature=temperature, top_p=top_p, entry_length=entry_length)
  print(x[0])

In [15]:
text_generation(prueba, temperature=1., top_p=0.8, entry_length=150)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:55<00:00, 55.11s/it]


No sé, dicen que es invierno
y que fuera está nevando.
Pero aquí sigue siendo verano.

Tú que te coronaste en Cotos sin necesidad de vestido,
o de trampas en papel de efecto retardado,
querías saber lo que me hizo el sol en el Atlántico
pero tú te quemaste la espalda
de las plumas de oro que esconden los sombreros.
Ni siquiera sabes lo que me hice. 

¡Que Dios te perdone! ¿Qué digo? Yo, por tu parte, me quedo contigo. 

Ven conmigo, por cierto, pues, adiós. 

Pero ya no soy quien para pedirte perdón. 

Entonces
lo sé, tú a tu vez, aunque no lo creas,
yo te echo de menos,
de ahí el pudor que me sube a la garganta.
Nunca, sin embargo,
sé que no volveré a verte,
nunca te olvido de nada. 

Yo, sin embargo, nunca volveré a verte. 

Hasta que no te duermas,
queda siempre allí, silencioso<|endoftext|>


In [16]:
prueba = """
Es porque aún somos pequeños.

Tú no dejabas de mirar por la ventana,
como si las golondrinas fueran a bailar por ti.
Irradiabas el aura y la luz de la vida
la mirada de una niña que ha visto más de lo que quería
por esa ventana que mostraba en el atardecer un nuevo alba.
"""

In [17]:
text_generation(prueba, temperature=0.9, top_p=0.8, entry_length=120)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:40<00:00, 40.58s/it]


Es porque aún somos pequeños.

Tú no dejabas de mirar por la ventana,
como si las golondrinas fueran a bailar por ti.
Irradiabas el aura y la luz de la vida
la mirada de una niña que ha visto más de lo que quería
por esa ventana que mostraba en el atardecer un nuevo alba.
La tristeza de la muerte
la espera del momento en que la luz es el símbolo de su fe. 

Tú has sido siempre una niña. 

Tú no estás ahí. 

Tú lo has sido siempre. 

Tú has estado siempre ahí. 

Tú no estabas ahí. 

Yo no estabas. 

Tú no estabas. 

Yo no estaba. 

Yo estaba. 

Yo estaba. 

Tú estabas. 

Tú no estabas. 

Tú no estabas.
Tú no estabas.
Yo no estabas. 

Yo no estabas<|endoftext|>


In [22]:
prueba = """El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
"""
text_generation(prueba, temperature=1., top_p=0.8, entry_length=90)


100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.44s/it]

El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
como en un ritual silencioso
al que ninguna de ellas había realizado,
medíganes, hadas, castas,
las delicias de la calle y la vista
del paisaje
una flor, florecillas de su perfume
y un riquísimo florecer. "
Vivía entonces en la plaza,
en un palacio cuyas puertas nunca se abrían,
cuando el automóvil cruzaba las puertas
como un hombre que se desata<|endoftext|>


In [23]:
text_generation(prueba, temperature=1., top_p=0.8, entry_length=90)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.41s/it]

El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
que nos rodeaba. No quería que nos cruzaran
sus ojos. Atrás quedaba, un campo que

aprendía ser bosque. 

Los viajeros pasaban de largo. Nos desplazábamos
avanzando. Los que nos seguían
todos se perdían. Pero yo,
capullo por la nieve
y gemía en mis adentros: 

—¡Ni en las cimas ni en las ramas! 

Mis pies se habían<|endoftext|>


In [24]:
text_generation(prueba, temperature=1., top_p=0.8, entry_length=90)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.69s/it]

El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
con los ojos cerrados.
Me adelanté a trompicones,
de repente se interrumpió el paisaje
y con el vaivén de la corriente
tengo la frente adherida al pavimento
de nubes de nieve y
tengo la cabeza contra la ventana
y la mirada en el techo
y por el cristal
todas las figuras que se mueven
que se mueven
Que giran y vuelan
Yo sé que las imágenes se van<|endoftext|>


In [25]:
text_generation(prueba, temperature=1., top_p=0.8, entry_length=90)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.90s/it]

El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
en vano. El aire,
perdido de sus vagones,
destrozaba la expresión.
Niego, y luego otro.
Por fin, el tren se detenía,
se detenía. Entonces, un nuevo,
un nuevo, estaba en movimiento,
buscaba una aguja. 

Pero ahora yo veía. 

Ya era la hora. 

La mañana, la hora

de dejar la estación, la hora<|endoftext|>


In [26]:
text_generation(prueba, temperature=1., top_p=0.8, entry_length=90)

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:17<00:00, 17.62s/it]

El tren repiqueteaba,
andaba lento, suave,
meciendo a los pasajeros
que parados, veían viajar al paisaje
para ver a la luna
sus estrellas. En el oscuro túnel,
había pájaros, peces, palomas,
todos revoloteaban. Y, aunque la luna
parecía alta,
si llegaba a su destino,
me imaginaba a la luna. 

Estaba ya en el umbral de la estación,
en la parte de atrás, en la casa,
en las escaleras, y sólo podía verlo. Y entonces, mientras corría
bajo<|endoftext|>
